In [10]:
import boto3, re, sys, math, json, os, sagemaker, urllib.request
import io
import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
from pyathena import connect
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, \
RepeatedStratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_curve, auc, mean_squared_error,\
precision_score, recall_score, f1_score, accuracy_score,\
confusion_matrix, plot_confusion_matrix, classification_report
from sagemaker.tuner import HyperparameterTuner
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import loguniform
import warnings
warnings.filterwarnings('ignore')


In [11]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [12]:
ingest_create_athena_db_passed = False

In [13]:
# set a database name
database_name = "ecommerce"

In [14]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [66]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [16]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS ecommerce


""


In [17]:
#S3 bucket name: olistteam5
#S3 URL: s3://olistteam5/ecommerce/

In [18]:
olist_dir='s3://olistteam5/ecommerce/'

In [19]:
## SQL statement to execute the analyte olist e-commerce table

table_name ='olist_customers_dataset'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name}', conn)


create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name}(
                customer_id String,
                customer_unique_id STRING,
                customer_zip_code_prefix int,
                customer_city string,
                customer_state string
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{olist_dir}/{table_name}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""
pd.read_sql(create_table, conn)
pd.read_sql(f'SELECT * FROM ecommerce.olist_customers_dataset LIMIT 5', conn)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,"""06b8999e2fba1a1fbc88172c00ba8bc7""","""861eff4711a542e4b93843c6dd7febb0""",None,franca,SP
1,"""18955e83d337fd6b2def6b18a428ac77""","""290c77bc529b7ac935b93aa66c333dc3""",None,sao bernardo do campo,SP
2,"""4e7b3e00288586ebd08712fdd0374a03""","""060e732b5b29e8181a18229c7b0b2b5e""",None,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,"""259dac757896d24d7702b9acbbff3f3c""",None,mogi das cruzes,SP
4,"""4f2d8ab171c80ec8364f7c12e35b23ad""","""345ecd01c38d18a9036ed96c73b8d066""",None,campinas,SP


In [20]:
table_name ='olist_geolocation_dataset'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name}', conn)


create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name}(
                geolocation_zip_code_prefix String,
                geolocation_lat STRING,
                geolocation_lng string,
                geolocation_city string,
                geolocation_state string
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{olist_dir}/{table_name}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""
pd.read_sql(create_table, conn)
pd.read_sql(f'SELECT * FROM ecommerce.olist_geolocation_dataset LIMIT 5', conn)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,"""26560""",-22.77030144304157,-43.42021375653865,mesquita,RJ
1,"""26553""",-22.793705182939505,-43.4303212864339,mesquita,RJ
2,"""26562""",-22.771253073382532,-43.42162442767146,mesquita,RJ
3,"""26540""",-22.80687957688535,-43.40939193182337,nilopolis,RJ
4,"""26515""",-22.819200221972213,-43.426500171830796,nilopolis,RJ


In [94]:
table_name ='olist_order_items_dataset'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name}', conn)


create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name}(
                order_id string,
                order_item_id string,
                product_id STRING,
                seller_id STRING,
                shipping_limit_date timestamp,
                price float,
                freight_value float
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{olist_dir}/{table_name}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""
pd.read_sql(create_table, conn)
pd.read_sql(f'SELECT * FROM ecommerce.olist_order_items_dataset LIMIT 5', conn)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,"""00010242fe8c5a6d1ba2dd792cb16214""",1,"""4244733e06e7ecb4970a6e2683c13e61""","""48436dade18ac8b2bce089ec2a041202""",2017-09-19 09:45:35,58.90,13.29
1,"""00018f77f2f0320c557190d7a144bdd3""",1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,"""000229ec398224ef6ca0657da4fc703e""",1,c777355d18b72b67abbeef9df44fd0fd,"""5b51032eddd242adc84c38acab88f23d""",2018-01-18 14:48:30,199.00,17.87
3,"""00024acbcdf0a6daa1e931b038114c75""",1,"""7634da152a4610f1595efa32f14722fc""","""9d7a1d34a5052409006425275ba1c2b4""",2018-08-15 10:10:18,12.99,12.79
4,"""00042b26cf59d7ce69dfabb4e55b4fd9""",1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [22]:
table_name ='olist_order_payments_dataset'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name}', conn)


create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name}(
                order_id String,
                payment_sequential int,
                payment_type string,
                payment_installments int,
                payment_value float
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{olist_dir}/{table_name}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""
pd.read_sql(create_table, conn)
pd.read_sql(f'SELECT * FROM ecommerce.olist_order_payments_dataset LIMIT 5', conn)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,"""25e8ea4e93396b6fa0d3dd708e76c1bd""",1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,"""42fdf880ba16b47b59251dd489d4441a""",1,credit_card,2,128.45


In [153]:
table_name ='olist_order_reviews_dataset'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name}', conn)


create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name}(
                review_id String,
                order_id string,
                review_score int,
                review_comment_title string,
                review_comment_message string,
                review_creation_date string,
                review_answer_timestamp string
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{olist_dir}/{table_name}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""
pd.read_sql(create_table, conn)
pd.read_sql(f'SELECT * FROM ecommerce.olist_order_reviews_dataset LIMIT 5', conn)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,"""7bc2406110b926393aa56f80a40eba40""","""73fc7af87114b39712e6da79b0a377eb""",4,,,2018-01-18 00:00:00,2018-01-18 21:46:59
1,"""80e641a11e56f04c1ad469d5645fdfde""","""a548910a1c6147796b98fdf73dbeba33""",5,,,2018-03-10 00:00:00,2018-03-11 03:05:13
2,"""228ce5500dc1d8e020d8d1322874b6f0""","""f9e4b658b201a9f2ecdecbb34bed034b""",5,,,2018-02-17 00:00:00,2018-02-18 14:36:24
3,"""e64fb393e7b32834bb789ff8bb30750e""","""658677c97b385a9be170737859d3511b""",5,,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,"""f7c4243c7fe1938f181bec41a392bdeb""","""8e6bfb81e283fa7e4f11123a3fb894f1""",5,,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [154]:
table_name ='olist_orders_dataset'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name}', conn)


create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name}(
                order_id string,
                customer_id string,
                order_status string,
                order_purchase_timestamp string,
                order_approved_at string,
                order_delivered_carrier_date string,
                order_delivered_customer_date string,
                order_estimated_delivery_date string
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{olist_dir}/{table_name}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""
pd.read_sql(create_table, conn)
pd.read_sql(f'SELECT * FROM ecommerce.olist_orders_dataset LIMIT 5', conn)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,"""9ef432eb6251297304e76186b10a928d""",delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,"""53cdb2fc8bc7dce0b6741e2150273451""",b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,"""47770eb9100c2d0c44946d9cf07ec65d""","""41ce2a54c0b03bf3443c3d931a367089""",delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,"""949d5b44dbf5de918fe9c16f97b45f8a""",f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,"""8ab97904e6daea8866dbdbc4fb7aad2c""",delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [155]:
table_name ='olist_products_dataset'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name}', conn)


create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name}(
                product_id string,
                product_category_name string,
                product_name_lenght float,
                product_description_lenght float,
                product_photos_qty float,
                product_weight_g float,
                product_length_cm float
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{olist_dir}/{table_name}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""
pd.read_sql(create_table, conn)
pd.read_sql(f'SELECT * FROM ecommerce.olist_products_dataset LIMIT 5', conn)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm
0,"""1e9e8ef04dbcff4541ed26657ea517e5""",perfumaria,40.0,287.0,1.0,225.0,16.0
1,"""3aa071139cb16b67ca9e5dea641aaa2f""",artes,44.0,276.0,1.0,1000.0,30.0
2,"""96bd76ec8810374ed1b65e291975717f""",esporte_lazer,46.0,250.0,1.0,154.0,18.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0
4,"""9dc1a7de274444849c219cff195d0b71""",utilidades_domesticas,37.0,402.0,4.0,625.0,20.0


In [128]:
table_name ='olist_sellers_dataset'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name}', conn)


create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name}(
                seller_id string,
                seller_zip_code_prefix int,
                seller_city string,
                seller_state string
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{olist_dir}/{table_name}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""
pd.read_sql(create_table, conn)
pd.read_sql(f'SELECT * FROM ecommerce.olist_sellers_dataset LIMIT 5', conn)

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,"""3442f8959a84dea7ee197c632cb2df15""",None,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,None,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,None,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,None,sao paulo,SP
4,"""51a04a8a6bdcb23deccc82b0b80742cf""",None,braganca paulista,SP


In [129]:
table_name ='product_category_name_translation'
pd.read_sql(f'DROP TABLE IF EXISTS {database_name}.{table_name}', conn)


create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name}(
                product_category_name string,
                product_category_name_english string
                )
                
                ROW FORMAT DELIMITED
                FIELDS TERMINATED BY ','
                LOCATION '{olist_dir}/{table_name}'
                TBLPROPERTIES ('skip.header.line.count'='1')
"""
pd.read_sql(create_table, conn)
pd.read_sql(f'SELECT * FROM ecommerce.product_category_name_translation LIMIT 5', conn)

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [133]:
statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,default
1,dsoaws
2,ecommerce


In [134]:
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

In [135]:
%store ingest_create_athena_db_passed

Stored 'ingest_create_athena_db_passed' (bool)


In [28]:
#Verify The Table Has Been Created Succesfully
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,tab_name
0,olist_customers_dataset
1,olist_geolocation_dataset
2,olist_order_items_dataset
3,olist_order_payments_dataset
4,olist_order_reviews_dataset


In [31]:
if table_name in df_show.values:
    ingest_create_athena_table_passed = True

In [32]:
%store ingest_create_athena_table_passed

Stored 'ingest_create_athena_table_passed' (bool)


In [36]:
#run a sample query
df=pd.read_sql(f'SELECT * FROM ecommerce.olist_order_items_dataset where price>60 LIMIT 5', conn)
df.head(5)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,"""00018f77f2f0320c557190d7a144bdd3""",1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
1,"""000229ec398224ef6ca0657da4fc703e""",1,c777355d18b72b67abbeef9df44fd0fd,"""5b51032eddd242adc84c38acab88f23d""",2018-01-18 14:48:30,199.00,17.87
2,"""00042b26cf59d7ce69dfabb4e55b4fd9""",1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
3,"""000576fe39319847cbb9d288c5617fa6""",1,"""557d850972a7d6f792fd18ae1400d9b6""","""5996cddab893a4652a15592fb58ab8db""",2018-07-10 12:30:45,810.00,70.75
4,"""0005a1a1728c9d785b8e2b08b904576c""",1,"""310ae3c140ff94b03219ad0adc3c778f""",a416b6a846a11724393025641d4edd5e,2018-03-26 18:31:29,145.95,11.65


In [37]:
#Review the New Athena Table in the Glue Catalog
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#">AWS Glue Catalog</a></b>'.format(
            region
        )
    )
)

In [39]:
ingest_create_athena_table_parquet_passed = False
%store -r ingest_create_athena_table_passed

In [149]:
#Create Parquet Files from CSV Table
# Set S3 path to Parquet data
from pyathena import connect
s3_path_parquet = "s3://olistteam5/parquet/".format(bucket)


# Set Athena parameters
database_name = "ecommerce"
table_name = "olist_order_items_dataset"
table_name_parquet = "olist_order_items_dataset_parquet"

In [150]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [151]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [152]:
# SQL statement to execute
statement = """CREATE TABLE IF NOT EXISTS {}.{}
WITH (format = 'PARQUET', external_location = '{}') AS
SELECT * FROM {}.{}""".format(
    database_name, table_name_parquet, s3_path_parquet, database_name, table_name
)


pd.read_sql(statement, conn)

,rows


In [153]:
statement = "MSCK REPAIR TABLE {}.{}".format(database_name, table_name_parquet)

df_partitions = pd.read_sql(statement, conn)
df_partitions.head(5)

""


In [156]:
statement = "SHOW TABLES in {}".format(database_name)
df_tables = pd.read_sql(statement, conn)
df_tables.head(5)

,tab_name
0,olist_customers_dataset
1,olist_geolocation_dataset
2,olist_order_items_dataset
3,olist_order_items_dataset_parquet
4,olist_order_payments_dataset


In [157]:
if table_name_parquet in df_tables.values:
    ingest_create_athena_table_parquet_passed = True

In [158]:
%store ingest_create_athena_table_parquet_passed

Stored 'ingest_create_athena_table_parquet_passed' (bool)


In [161]:

statement = """SELECT * FROM {}.{} LIMIT 100""".format(
    database_name, table_name_parquet
)

df = pd.read_sql(statement, conn)
df.head(5)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,ffa378a0d8e871367fbe51bdd9e12a1f,1,"""6a83dafadbcf4112388e8173860dd3b7""","""888faa8bfb0b159c37de6d898b961c31""",2017-09-28 20:30:15,42.69,16.11
1,ffa39020fe7c8a3e907320e1bec4b985,1,"""9e0e152552a1323f7e5dcf63d50cdae3""",cc419e0650a3c5ba77189a1882b7556a,2017-11-10 18:55:13,56.99,14.15
2,ffa466d3e55f98477b7e0da764df3c6c,1,"""07ffc018eaf23e086370dea42c74077b""","""49067458c68f7701fd334ce326accbe0""",2018-05-22 08:30:44,28.45,18.23
3,ffa466d3e55f98477b7e0da764df3c6c,2,"""53c61580d8e5c4941a358ee0335c1d5c""","""49067458c68f7701fd334ce326accbe0""",2018-05-22 08:30:44,28.45,18.23
4,ffa4713aab0ab633f0f6daa12baf90cb,1,"""64fb265487de2238627ce43fe8a67efc""","""4a3ca9315b744ce9f8e9374361493884""",2018-07-23 17:23:22,79.90,26.86


In [162]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#">AWS Glue Catalog</a></b>'.format(
            region
        )
    )
)